In [1]:
from dao.email import AVAILABLE_EMAIL_DAOS
from typing import List

In [2]:
from dao.email import DAOEmailGenerated
from models.email import EmailGeneratedInDB

dao_generated_emails = DAOEmailGenerated()
all_generated: List[EmailGeneratedInDB]= dao_generated_emails.find_all()
ids = [email.og_doc_id for email in all_generated]

In [ ]:
from tqdm import tqdm
from models.email import EmailGenerated, EmailGmailInDB
from services.email_generator import generate_email_based_on_og_email
from analysis.nlp_transformations import clean_text, separate_previous_conversation, remove_footers

gmail_daos_info = [('gmail1', AVAILABLE_EMAIL_DAOS['gmail1']), ('gmail2', AVAILABLE_EMAIL_DAOS['gmail2']), ('gmail3', AVAILABLE_EMAIL_DAOS['gmail3'])]



for gmail_dao_name, gmail_dao in gmail_daos_info:
    emails: List[EmailGmailInDB] = gmail_dao.find_many_by_query({
        'email_labels': {'$not':{'$regex': "Kategori"}},
        "_id": {
            "$nin": ids
        }
    })
    print(gmail_dao_name)
    for email in tqdm(emails, total=len(emails), desc=f'Generating emails for {gmail_dao_name}', unit='emails', miniters=1):
        subject = email.subject
        if email.text_plain is None:
            continue
        email_text, _ = separate_previous_conversation(email.text_plain)
        email_text = remove_footers(clean_text(email_text))
        if len(email_text) < 10:
            continue
        try:
            generated_text = generate_email_based_on_og_email(subject, email_text, email.detected_language)
        except ValueError:
            continue
        except ConnectionError:
            continue
        email_generated = EmailGenerated(
            og_db_name=gmail_dao_name,
            og_doc_id=email.id,
            subject=subject,
            text_plain=generated_text,
            language=email.detected_language,
        )
        dao_generated_emails.insert_one(email_generated)


gmail1


Generating emails for gmail1:   3%|▎         | 13/461 [00:32<17:26,  2.34s/emails]